In [16]:
import numpy as np
import bqplot.pyplot as plt
from bqplot import (
    CATEGORY10,
    ColorScale,
    DateScale,
    LinearScale,
    Lines,
    Axis,
    Figure,
    OrdinalScale,
    Bars,
    Axis,
)
from lib.ENTSOE import fetch_production, fetch_price, ENTSOE_DOMAIN_MAPPINGS
import pandas as pd
from datetime import datetime, timedelta
from pkgutil import iter_modules
import importlib
import os
from ipywidgets import (
    TwoByTwoLayout,
    Button,
    Layout,
    jslink,
    IntText,
    IntSlider,
    FloatSlider,
    Dropdown,
)
import json
import random
import requests
from ipyleaflet import Map, GeoJSON, WidgetControl, FullScreenControl, ZoomControl, Choropleth
import ipywidgets as widgets
from ipyflex import FlexLayout
from branca.colormap import linear

# This is completely stupid, but at least it will prevent bots from getting the key (like copilot)
passwd = "Jupyter is super amazing, one of the coolest tool out there"
key = '~¦Ö²©\x95×V\x96£TÖ§\x9d\x99ÕWÄ\x9a\x99ÝÌÏ\x94]VÒ§Ë\x83¨Ì\x85¨\x99Ê'
actual_key = []
for i in range(len(key)):
    actual_key.append(chr(ord(key[i]) - ord(passwd[i])))
os.environ["ENTSOE_TOKEN"] = "".join(actual_key)

with open("./eu-countries.geo.json") as f:
    countries = json.load(f)

    # Filter countries that are not supported by lib.ENTSOE
    supported_countries = ENTSOE_DOMAIN_MAPPINGS
    countries["features"] = [
        feature
        for feature in countries["features"]
        if feature["properties"]["iso_a2"] in supported_countries
    ]

# some countries dont get proper return from fetch_production() so we remove them
iso_codes_to_remove = {"AL", "BY", "GB", "RU", "RS", "UA"}

countries["features"] = [
    feature
    for feature in countries["features"]
    if feature["properties"]["iso_a2"] not in iso_codes_to_remove
]


# get production values for supported countries
prod_list = []
for entry in countries["features"]:
    prod = fetch_production(entry["properties"]["iso_a2"])
    prod_list.append(prod)


# create dict with total production values per country
total_prod_dict = {}
for prod_return in prod_list:
    # skip empty entries
    if not prod_return:
        continue

    total_production = 0.0
    for entry in prod_return:
        production_values = entry['production']
        total_production += sum(value for value in production_values.values() if value is not None)
        total_prod_dict[entry['zoneKey']] = total_production

choro = Choropleth(
    geo_data=countries,
    choro_data=total_prod_dict,
    colormap=linear.YlOrRd_04,
    border_color='black',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'}
)

m = Map(
    zoom=3,
    center=(48.30, 23.23),
    zoom_control=False,
    layout=dict(height="98%", width="98%"),
)
m.add(ZoomControl(position="topright"))
m.add(FullScreenControl())
m.add(choro)



Required generation type coal is missing from GR
Required generation type coal is missing from GR
Required generation type coal is missing from IT


Map(center=[48.3, 23.23], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright')…

In [19]:
w = {"Map": m}
header = dict(title="Electricity Production", buttons=[])
FlexLayout(
    w,
    style={"height": "calc(100vh - 60px)"},
    header=header,
    template="Electricity_map.json",
    editable=False,
)

FlexLayout(children={'Map': Map(bottom=723.0, center=[48.3, 23.23], controls=(AttributionControl(options=['pos…